In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

# Any results you write to the current directory are saved as output.

In [4]:
data=pd.read_csv("../input/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')

In [5]:
data=data[data['source']=='ted']

In [6]:
data.head()

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,ted,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
3,ted,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है


In [7]:
data.drop_duplicates(inplace=True)

* ### Let us pick any 25000 rows from the dataset.

In [8]:
data = data.sample(n = 20000 ,random_state = 42)

In [9]:
data['english_sentence']=data['english_sentence'].apply(lambda x: x.lower())
data['hindi_sentence']=data['hindi_sentence'].apply(lambda x: x.lower())

In [10]:
import string

In [11]:
translator = str.maketrans('', '', string.punctuation)
data['english_sentence'] = data['english_sentence'].apply(lambda x: x.translate(translator))
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: x.translate(translator))

In [12]:
remove_digits = str.maketrans('', '', digits)
data['english_sentence']=data['english_sentence'].apply(lambda x: x.translate(remove_digits))
data['hindi_sentence']=data['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

data['english_sentence']=data['english_sentence'].apply(lambda x: x.strip())
data['hindi_sentence']=data['hindi_sentence'].apply(lambda x: x.strip())
data['english_sentence']=data['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
data['hindi_sentence']=data['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))


In [13]:
# Add start and end tokens to target sequences
data['hindi_sentence'] = data['hindi_sentence'].apply(lambda x : '$ '+ x + ' %')

In [14]:
data.head()

,source,english_sentence,hindi_sentence
82040,ted,we still dont know who her parents are who she is,$ हम अभी तक नहीं जानते हैं कि उसके मातापिता कौ...
85038,ted,no keyboard,$ कोई कुंजीपटल नहीं %
58018,ted,but as far as being a performer,$ लेकिन एक कलाकार होने के साथ %
74470,ted,and this particular balloon,$ और यह खास गुब्बारा %
122330,ted,and its not as hard as you think integrate cli...,$ और जितना आपको लगता है यह उतना कठिन नहीं हैअप...


In [15]:
### Get English and Hindi Vocabulary
eng_word_corpus=set()
for eng in data['english_sentence']:
    for word in eng.split():
        if word not in eng_word_corpus:
            eng_word_corpus.add(word)

hindi_word_corpus=set()
for hin in data['hindi_sentence']:
    for word in hin.split():
        if word not in hindi_word_corpus:
            hindi_word_corpus.add(word)

In [16]:
data['length_eng_sentence']=data['english_sentence'].apply(lambda x:len(x.split(" ")))
data['length_hin_sentence']=data['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [17]:
data=data[data['length_eng_sentence']<=20]
data=data[data['length_hin_sentence']<=20]

In [18]:
data.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
82040,ted,we still dont know who her parents are who she is,$ हम अभी तक नहीं जानते हैं कि उसके मातापिता कौ...,11,16
85038,ted,no keyboard,$ कोई कुंजीपटल नहीं %,2,5
58018,ted,but as far as being a performer,$ लेकिन एक कलाकार होने के साथ %,7,8
74470,ted,and this particular balloon,$ और यह खास गुब्बारा %,4,6
122330,ted,and its not as hard as you think integrate cli...,$ और जितना आपको लगता है यह उतना कठिन नहीं हैअप...,16,20


In [19]:
max_length_src=max(data['length_hin_sentence'])
max_length_tar=max(data['length_eng_sentence'])

In [20]:
input_words = sorted(list(eng_word_corpus))
target_words = sorted(list(hindi_word_corpus))
num_encoder_tokens = len(eng_word_corpus)
num_decoder_tokens = len(hindi_word_corpus)
num_encoder_tokens, num_decoder_tokens

(12548, 15514)

In [21]:
num_decoder_tokens+=1

In [22]:
input_token_index = {}
for i, word in enumerate(input_words):
    input_token_index[word] = i + 1

target_token_index = {}
for i, word in enumerate(target_words):
    target_token_index[word] = i + 1

In [23]:
reverse_input_char_index = {}
for word, i in input_token_index.items():
    reverse_input_char_index[i] = word

reverse_target_char_index = {}
for word, i in target_token_index.items():
    reverse_target_char_index[i] = word

### Split the data into train and test

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X = data['english_sentence']
y = data['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((15861,), (3966,))

### Let us save this data

In [26]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [27]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


In [28]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [29]:
latent_dim=300

In [30]:
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [31]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [32]:
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    3764400     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    4654500     input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [33]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy' , metrics = ['accuracy'])

In [34]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

In [35]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/50
123/123 [==============================] - 62s 504ms/step - loss: 6.4792 - acc: 0.1301 - val_loss: 6.2055 - val_acc: 0.1413
Epoch 2/50
123/123 [==============================] - 57s 463ms/step - loss: 5.9250 - acc: 0.1527 - val_loss: 5.9176 - val_acc: 0.1688
Epoch 3/50
123/123 [==============================] - 57s 463ms/step - loss: 5.5846 - acc: 0.1841 - val_loss: 5.7287 - val_acc: 0.1956
Epoch 4/50
123/123 [==============================] - 57s 464ms/step - loss: 5.3360 - acc: 0.2051 - val_loss: 5.6026 - val_acc: 0.2094
Epoch 5/50
123/123 [==============================] - 57s 464ms/step - loss: 5.1273 - acc: 0.2217 - val_loss: 5.5381 - val_acc: 0.2206
Epoch 6/50
123/123 [==============================] - 57s 462ms/step - loss: 4.9398 - acc: 0.2361 - val_loss: 5.4895 - val_acc: 0.2288
Epoch 7/50
123/123 [==============================] - 57s 461ms/

In [36]:
model.save_weights('nmt_weights.h5')

In [37]:
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [38]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['$']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        if (sampled_char == '%' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [39]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [40]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: of a whole bunch of little particles
Actual Hindi Translation: ोटी चीजों से बना 
Predicted Hindi Translation:  कम से कि लोग सर 


In [41]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: no one was a hero no one was a hero
Actual Hindi Translation: भी नायक नहीं था कोई भी नायक नहीं 
Predicted Hindi Translation:  कोई भी नायक नहीं था कोई भी कोई ज़रुरत नहीं 


In [42]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: and about three years ago
Actual Hindi Translation: गभग तीन साल पह
Predicted Hindi Translation:  और लगभग तीन साल पह
